In [9]:
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np

from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

########################
# 1) Device Setup      #
########################
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

########################
# 2) Data + Categories #
########################

category_columns = [
    "Unlawful detention",
    "Human trafficking",
    "Enslavement",
    "Willful killing of civilians",
    "Mass execution",
    "Kidnapping",
    "Extrajudicial killing",
    "Forced disappearance",
    "Damage or destruction of civilian critical infrastructure",
    "Damage or destruction, looting, or theft of cultural heritage",
    "Military operations (battle, shelling)",
    "Gender-based or other conflict-related sexual violence",
    "Violent crackdowns on protesters/opponents/civil rights abuse",
    "Indiscriminate use of weapons",
    "Torture or indications of torture",
    "Persecution based on political, racial, ethnic, gender, or sexual orientation",
    "Movement of military, paramilitary, or other troops and equipment"
]

# Load CSVs (adjust to your actual file paths)
train_df = pd.read_csv("train.csv")
val_df   = pd.read_csv("val.csv")
test_df  = pd.read_csv("test.csv")

##################################
# 3) Custom Dataset for Articles #
##################################

class ArticleDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.float)
        return item

    def __len__(self):
        return len(self.labels)

###############################
# 4) Tokenization + Datasets #
###############################

# Load the tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Tokenize (Incident Narrative column)
train_encodings = tokenizer(
    list(train_df["Incident Narrative"].values),
    truncation=True,
    padding=True
)
val_encodings   = tokenizer(
    list(val_df["Incident Narrative"].values),
    truncation=True,
    padding=True
)
test_encodings  = tokenizer(
    list(test_df["Incident Narrative"].values),
    truncation=True,
    padding=True
)

# Extract labels (multi-hot vectors for each category)
train_labels = train_df[category_columns].values
val_labels   = val_df[category_columns].values
test_labels  = test_df[category_columns].values

# Create Dataset objects
train_dataset = ArticleDataset(train_encodings, train_labels)
val_dataset   = ArticleDataset(val_encodings, val_labels)
test_dataset  = ArticleDataset(test_encodings, test_labels)

########################
# 5) Model + Training  #
########################

# num_labels = number of category columns
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=len(category_columns)
)

# Move the model to the correct device
model.to(device)

# Define compute_metrics for multi-label classification
def compute_metrics(p):
    # p.predictions are logits; p.label_ids are ground truth
    preds = torch.sigmoid(torch.tensor(p.predictions))  # Convert logits to probabilities
    preds = (preds > 0.5).int().cpu().numpy()
    labels = torch.tensor(p.label_ids).cpu().numpy()

    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, preds, average='weighted'
    )
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# Training arguments
training_args = TrainingArguments(
    output_dir='./results',         # Output directory
    eval_strategy="epoch",          # Evaluate at the end of each epoch
    save_strategy="epoch",          # Save model at the end of each epoch
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=30,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1",     # Use F1 score for best model
    logging_dir='./logs'
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()

# Save the model
trainer.save_model("./bert-multiclass-model")

###########################################
# 6) Single-Category Inference on Test Set
###########################################

# Create a DataLoader for the test dataset
test_loader = DataLoader(
    test_dataset,
    batch_size=16,  # Adjust as appropriate
    shuffle=False
)

model.eval()  # evaluation mode

y_true = []
y_pred = []

with torch.no_grad():
    for batch in test_loader:
        # Move batch inputs/labels to the same device as the model
        inputs = {k: v.to(device) for k, v in batch.items() if k != 'labels'}
        labels = batch['labels'].to(device)

        outputs = model(**inputs)   # forward pass
        logits = outputs.logits     # shape: (batch_size, 17)

        # Convert to numpy
        probs = logits.detach().cpu().numpy()

        y_true.extend(labels.detach().cpu().numpy())
        y_pred.extend(probs)

# Convert to numpy arrays
y_true = np.array(y_true)  # shape: (num_samples, 17)
y_pred = np.array(y_pred)  # shape: (num_samples, 17)

# 1) Single-label by argmax
y_pred_single = np.argmax(y_pred, axis=1)  # shape: (num_samples,)

# 2) Exact-match accuracy if predicted category is among the "1"s in ground truth
accuracy = np.mean([
    1 if y_true[i, y_pred_single[i]] == 1 else 0
    for i in range(len(y_true))
])

print("Exact-match single-category accuracy:", accuracy)


Using device: cuda


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  3%|▎         | 21/630 [00:13<05:03,  2.01it/s]c:\Users\ymasr\OneDrive\Desktop\projects\BERT_GMU_Classiifcation\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\ymasr\OneDrive\Desktop\projects\BERT_GMU_Classiifcation\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_

{'eval_loss': 0.46326664090156555, 'eval_accuracy': 0.0, 'eval_f1': 0.0287856071964018, 'eval_precision': 0.08347826086956522, 'eval_recall': 0.017391304347826087, 'eval_runtime': 0.7218, 'eval_samples_per_second': 78.967, 'eval_steps_per_second': 5.542, 'epoch': 1.0}


  7%|▋         | 42/630 [00:28<04:46,  2.05it/s]c:\Users\ymasr\OneDrive\Desktop\projects\BERT_GMU_Classiifcation\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\ymasr\OneDrive\Desktop\projects\BERT_GMU_Classiifcation\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\ymasr\OneDrive\Desktop\projects\BERT_GMU_Classiifcation\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor pre

{'eval_loss': 0.35947632789611816, 'eval_accuracy': 0.08771929824561403, 'eval_f1': 0.3071107720993304, 'eval_precision': 0.31105304125430894, 'eval_recall': 0.3217391304347826, 'eval_runtime': 0.7053, 'eval_samples_per_second': 80.815, 'eval_steps_per_second': 5.671, 'epoch': 2.0}


...

{'eval_loss': 0.19533133506774902, 'eval_accuracy': 0.2982456140350877, 'eval_f1': 0.5902509014706716, 'eval_precision': 0.6205507246376811, 'eval_recall': 0.5826086956521739, 'eval_runtime': 0.7183, 'eval_samples_per_second': 79.358, 'eval_steps_per_second': 5.569, 'epoch': 29.0}


100%|██████████| 630/630 [09:36<00:00,  2.03it/s]c:\Users\ymasr\OneDrive\Desktop\projects\BERT_GMU_Classiifcation\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\ymasr\OneDrive\Desktop\projects\BERT_GMU_Classiifcation\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\ymasr\OneDrive\Desktop\projects\BERT_GMU_Classiifcation\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor pr

{'eval_loss': 0.1951710730791092, 'eval_accuracy': 0.2982456140350877, 'eval_f1': 0.5902509014706716, 'eval_precision': 0.6205507246376811, 'eval_recall': 0.5826086956521739, 'eval_runtime': 0.6712, 'eval_samples_per_second': 84.924, 'eval_steps_per_second': 5.96, 'epoch': 30.0}


100%|██████████| 630/630 [09:39<00:00,  1.09it/s]


{'train_runtime': 579.9261, 'train_samples_per_second': 16.709, 'train_steps_per_second': 1.086, 'train_loss': 0.1972462472461519, 'epoch': 30.0}
Exact-match single-category accuracy: 0.7674418604651163
